# New functions

These are recently written functions that have not made it into the main documentation

## Python Lesson: Errors and Exceptions

In [1]:
# you would normaly install eppy by doing #
# python setup.py install
# or
# pip install eppy
# or
# easy_install eppy

# if you have not done so, uncomment the following three lines
import sys
# pathnameto_eppy = 'c:/eppy'
pathnameto_eppy = '../'
sys.path.append(pathnameto_eppy) 


When things go wrong in your eppy script, you get "Errors and Exceptions". 

To know more about how this works in python and eppy, take a look at [Python: Errors and Exceptions](http://docs.python.org/2/tutorial/errors.html)

## Setting IDD name

When you work with Energyplus you are working with **idf** files (files that have the extension \*.idf). There is another file that is very important, called the **idd** file. This is the file that defines all the objects in Energyplus. Esch version of Energyplus has a different **idd** file. 

So eppy needs to know which **idd** file to use. Only one **idd** file can be used in a script or program. This means that you cannot change the **idd** file once you have selected it. Of course you have to first select an **idd** file before eppy can work.

If you use eppy and break the above rules, eppy will raise an exception. So let us use eppy incorrectly and make eppy raise the exception, just see how that happens.

First let us try to open an **idf** file without setting an **idd** file.

In [15]:
from eppy import modeleditor 
from eppy.modeleditor import IDF
fname1 = "../eppy/resources/idffiles/V_7_2/smallfile.idf"


Now let us open file fname1 without setting the **idd** file

In [16]:
try:
    idf1 = IDF(fname1)
except Exception as e:
    raise e
    

OK. It does not let you do that and it raises an exception

So let us set the **idd** file and then open the idf file

In [17]:
iddfile = "../eppy/resources/iddfiles/Energy+V7_2_0.idd"
IDF.setiddname(iddfile)
idf1 = IDF(fname1)


That worked without raising an exception

Now let us try to change the **idd** file. Eppy should not let you do this and should raise an exception.

In [18]:
try:
    IDF.setiddname("anotheridd.idd")
except Exception as e:
    raise e   
    

IDDAlreadySetError: IDD file is set to: ../eppy/resources/iddfiles/Energy+V7_2_0.idd

Excellent!! It raised the exception we were expecting.

## Check range for fields

The fields of idf objects often have a range of legal values. The following functions will let you discover what that range is and test if your value lies within that range

demonstrate two new functions:

- EpBunch.getrange(fieldname) # will return the ranges for that field
- EpBunch.checkrange(fieldname) # will throw an exception if the value is outside the range

In [19]:
from eppy import modeleditor 
from eppy.modeleditor import IDF
iddfile = "../eppy/resources/iddfiles/Energy+V7_2_0.idd"
fname1 = "../eppy/resources/idffiles/V_7_2/smallfile.idf"


In [20]:
# IDF.setiddname(iddfile)# idd ws set further up in this page
idf1 = IDF(fname1)


In [21]:
building = idf1.idfobjects['building'][0]
print(building)



BUILDING,
    Empire State Building,    !- Name
    30,                       !- North Axis
    City,                     !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days



In [22]:
print(building.getrange("Loads_Convergence_Tolerance_Value"))


{'maximum': 0.5, 'minimum': None, 'maximum<': None, 'minimum>': 0.0, 'type': 'real'}


In [23]:
print(building.checkrange("Loads_Convergence_Tolerance_Value"))


0.04


Let us set these values outside the range and see what happens

In [24]:
building.Loads_Convergence_Tolerance_Value = 0.6
from eppy.bunch_subclass import RangeError
try:
    print(building.checkrange("Loads_Convergence_Tolerance_Value"))
except RangeError as e:
    raise e
    

RangeError: Value 0.6 is not less or equal to the 'maximum' of 0.5

So the Range Check works

## Looping through all the fields in an idf object

We have seen how to check the range of field in the idf object. What if you want to do a *range check* on all the fields in an idf object ? To do this we will need a list of all the fields in the idf object. We can do this easily by the following line

In [25]:
print(building.fieldnames)


['key', 'Name', 'North_Axis', 'Terrain', 'Loads_Convergence_Tolerance_Value', 'Temperature_Convergence_Tolerance_Value', 'Solar_Distribution', 'Maximum_Number_of_Warmup_Days', 'Minimum_Number_of_Warmup_Days']


So let us use this

In [27]:
for fieldname in building.fieldnames:
    print("%s = %s" % (fieldname, building[fieldname]))
    

key = BUILDING
Name = Empire State Building
North_Axis = 30.0
Terrain = City
Loads_Convergence_Tolerance_Value = 0.6
Temperature_Convergence_Tolerance_Value = 0.4
Solar_Distribution = FullExterior
Maximum_Number_of_Warmup_Days = 25
Minimum_Number_of_Warmup_Days = 6


Now let us test if the values are in the legal range. We know that "Loads_Convergence_Tolerance_Value" is out of range

In [28]:
from eppy.bunch_subclass import RangeError
for fieldname in building.fieldnames:
    try:
        building.checkrange(fieldname)
        print("%s = %s #-in range" % (fieldname, building[fieldname],))
    except RangeError as e:
        print("%s = %s #-****OUT OF RANGE****" % (fieldname, building[fieldname],))
        

key = BUILDING #-in range
Name = Empire State Building #-in range
North_Axis = 30.0 #-in range
Terrain = City #-in range
Loads_Convergence_Tolerance_Value = 0.6 #-****OUT OF RANGE****
Temperature_Convergence_Tolerance_Value = 0.4 #-in range
Solar_Distribution = FullExterior #-in range
Maximum_Number_of_Warmup_Days = 25 #-in range
Minimum_Number_of_Warmup_Days = 6 #-in range


You see, we caught the out of range value

## Blank idf file

Until now in all our examples, we have been reading an idf file from disk:

- How do I create a blank new idf file  
- give it a file name
- Save it to the disk

Here are the steps to do that

In [29]:
# some initial steps
from eppy.modeleditor import IDF
iddfile = "../eppy/resources/iddfiles/Energy+V7_2_0.idd"
# IDF.setiddname(iddfile) # Has already been set 

# - Let us first open a file from the disk
fname1 = "../eppy/resources/idffiles/V_7_2/smallfile.idf"
idf_fromfilename = IDF(fname1) # initialize the IDF object with the file name

idf_fromfilename.printidf()



VERSION,
    7.3;                      !- Version Identifier

SIMULATIONCONTROL,
    Yes,                      !- Do Zone Sizing Calculation
    Yes,                      !- Do System Sizing Calculation
    Yes,                      !- Do Plant Sizing Calculation
    No,                       !- Run Simulation for Sizing Periods
    Yes;                      !- Run Simulation for Weather File Run Periods

BUILDING,
    Empire State Building,    !- Name
    30,                       !- North Axis
    City,                     !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days

SITE:LOCATION,
    CHICAGO_IL_USA TMY2-94846,    !- Name
    41.78,                    !- Latitude
    -87.75,                   !- Longit

In [30]:
# - now let us open a file from the disk differently
fname1 = "../eppy/resources/idffiles/V_7_2/smallfile.idf"
fhandle = open(fname1, 'r') # open the file for reading and assign it a file handle
idf_fromfilehandle = IDF(fhandle) # initialize the IDF object with the file handle

idf_fromfilehandle.printidf()



VERSION,
    7.3;                      !- Version Identifier

SIMULATIONCONTROL,
    Yes,                      !- Do Zone Sizing Calculation
    Yes,                      !- Do System Sizing Calculation
    Yes,                      !- Do Plant Sizing Calculation
    No,                       !- Run Simulation for Sizing Periods
    Yes;                      !- Run Simulation for Weather File Run Periods

BUILDING,
    Empire State Building,    !- Name
    30,                       !- North Axis
    City,                     !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days

SITE:LOCATION,
    CHICAGO_IL_USA TMY2-94846,    !- Name
    41.78,                    !- Latitude
    -87.75,                   !- Longit

In [32]:
# So IDF object can be initialized with either a file name or a file handle

# - How do I create a blank new idf file  
idftxt = "" # empty string
from io import StringIO
fhandle = StringIO(idftxt) # we can make a file handle of a string
idf_emptyfile = IDF(fhandle) # initialize the IDF object with the file handle

idf_emptyfile.printidf()


It did not print anything. Why should it. It was empty. 

What if we give it a string that was not blank

In [33]:
# - The string does not have to be blank
idftxt = "VERSION, 7.3;" # Not an emplty string. has just the version number
fhandle = StringIO(idftxt) # we can make a file handle of a string
idf_notemptyfile = IDF(fhandle) # initialize the IDF object with the file handle

idf_notemptyfile.printidf()



VERSION,
    7.3;                      !- Version Identifier



Aha !

Now let us give it a file name

In [34]:
# - give it a file name
idf_notemptyfile.idfname = "notemptyfile.idf"
# - Save it to the disk
idf_notemptyfile.save()


Let us confirm that the file was saved to disk

In [35]:
txt = open("notemptyfile.idf", 'r').read()# read the file from the disk
print(txt)


!- Darwin Line endings 

VERSION,
    7.3;                      !- Version Identifier


Yup ! that file was saved. Let us delete it since we were just playing

In [36]:
import os
os.remove("notemptyfile.idf")


## Deleting, copying/adding and making new idfobjects

### Making a new idf object

Let us start with a blank idf file and make some new "MATERIAL" objects in it

In [38]:
# making a blank idf object
blankstr = ""
from io import StringIO
idf = IDF(StringIO(blankstr))


To make and add a new idfobject object, we use the function IDF.newidfobject(). We want to make an object of type "MATERIAL"

In [39]:
newobject = idf.newidfobject("material") 

    

In [40]:
print(newobject)



MATERIAL,
    ,                         !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance



Let us give this a name, say "Shiny new material object"

In [41]:
newobject.Name = "Shiny new material object"
print(newobject)



MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance



In [42]:
anothermaterial = idf.newidfobject("material")
anothermaterial.Name = "Lousy material"
thirdmaterial = idf.newidfobject("material")
thirdmaterial.Name = "third material"
print(thirdmaterial)



MATERIAL,
    third material,           !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance



Let us look at all the "MATERIAL" objects

In [43]:
print(idf.idfobjects["MATERIAL"])


[
MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
MATERIAL,
    Lousy material,           !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
MATERIAL,
    third material,           !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    

As we can see there are three MATERIAL idfobjects. They are:

1. Shiny new material object
2. Lousy material
3. third material

### Deleting an idf object

Let us remove 2. Lousy material. It is the second material in the list. So let us remove the second material

In [44]:
idf.popidfobject('MATERIAL', 1) # first material is '0', second is '1'



MATERIAL,
    Lousy material,           !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

In [45]:
print(idf.idfobjects['MATERIAL'])


[
MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
MATERIAL,
    third material,           !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
]


You can see that the second material is gone ! Now let us remove the first material, but do it using a different function

In [46]:
firstmaterial = idf.idfobjects['MATERIAL'][-1]


In [47]:
idf.removeidfobject(firstmaterial)


In [48]:
print(idf.idfobjects['MATERIAL'])


[
MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
]


So we have two ways of deleting an idf object:

1. popidfobject -> give it the idf key: "MATERIAL", and the index number
2. removeidfobject -> give it the idf object to be deleted

### Copying/Adding an idf object

Having deleted two "MATERIAL" objects, we have only one left. Let us make a copy of this object and add it to our idf file

In [49]:
onlymaterial = idf.idfobjects["MATERIAL"][0]

In [50]:
idf.copyidfobject(onlymaterial)


MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

In [51]:
print(idf.idfobjects["MATERIAL"])

[
MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
]


So now we have a copy of the material. You can use this method to copy idf objects from other idf files too.

## Making an idf object with named arguments

What if we wanted to make an idf object with values for it's fields? We can do that too.

In [52]:
gypboard = idf.newidfobject('MATERIAL', Name="G01a 19mm gypsum board",
                            Roughness="MediumSmooth",
                            Thickness=0.019,
                            Conductivity=0.16,
                            Density=800,
                            Specific_Heat=1090)

In [53]:
print(gypboard)


MATERIAL,
    G01a 19mm gypsum board,    !- Name
    MediumSmooth,             !- Roughness
    0.019,                    !- Thickness
    0.16,                     !- Conductivity
    800,                      !- Density
    1090,                     !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance



newidfobject() also fills in the default values like "Thermal Absorptance", "Solar Absorptance", etc.

In [54]:
print(idf.idfobjects["MATERIAL"])

[
MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
MATERIAL,
    G01a 19mm gypsum board,    !- Name
    MediumSmooth,             !- Roughness
    0.019,                    !- Thickness

## Renaming an idf object

It is easy to rename an idf object. If we want to rename the gypboard object that we created above, we simply say:


But this could create a problem. What if this gypboard is part of a "CONSTRUCTION" object. The construction object will refer to the gypboard by name. If we change the name of the gypboard, we should change it in the construction object. 

But there may be many constructions objects using the gypboard. Now we will have to change it in all those construction objects. Sounds painfull. 

Let us try this with an example:

In [55]:
interiorwall = idf.newidfobject("CONSTRUCTION", Name="Interior Wall",
                 Outside_Layer="G01a 19mm gypsum board",
                 Layer_2="Shiny new material object",
                 Layer_3="G01a 19mm gypsum board")
print(interiorwall)



CONSTRUCTION,
    Interior Wall,            !- Name
    G01a 19mm gypsum board,    !- Outside Layer
    Shiny new material object,    !- Layer 2
    G01a 19mm gypsum board;    !- Layer 3



to rename gypboard and have that name change in all the places we call modeleditor.rename(idf, key, oldname, newname)

In [56]:
modeleditor.rename(idf, "MATERIAL", "G01a 19mm gypsum board", "peanut butter")


MATERIAL,
    peanut butter,            !- Name
    MediumSmooth,             !- Roughness
    0.019,                    !- Thickness
    0.16,                     !- Conductivity
    800,                      !- Density
    1090,                     !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

In [57]:
print(interiorwall)


CONSTRUCTION,
    Interior Wall,            !- Name
    peanut butter,            !- Outside Layer
    Shiny new material object,    !- Layer 2
    peanut butter;            !- Layer 3



Now we have "peanut butter" everywhere. At least where we need it. Let us look at the entir idf file, just to be sure

In [58]:
idf.printidf()


MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

MATERIAL,
    Shiny new material object,    !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

MATERIAL,
    peanut butter,            !- Name
    MediumSmooth,             !- Roughness
    0.019,                    !- Thickness
    0

### Turn off default values

Can I turn off the defautl values. Yes you can:

In [59]:
defaultmaterial = idf.newidfobject("MATERIAL", 
                                     Name='with default')
print(defaultmaterial)
nodefaultmaterial = idf.newidfobject("MATERIAL", 
                                     Name='Without default',
                                  defaultvalues=False)
print(nodefaultmaterial)


MATERIAL,
    with default,             !- Name
    ,                         !- Roughness
    ,                         !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance


MATERIAL,
    Without default;          !- Name



- But why would you want to turn it off. 
- Well .... sometimes you have to 
- Try it with the object `DAYLIGHTING:CONTROLS`, and you will see the need for `defaultvalues=False`

Of course, internally EnergyPlus will still use the default values it it is left blank. If just won't turn up in the IDF file.

## Zone area and volume

The idf file has zones with surfaces and windows. It is easy to get the attributes of the surfaces and windows as we have seen in the tutorial. Let us review this once more:


In [60]:
from eppy import modeleditor 
from eppy.modeleditor import IDF
iddfile = "../eppy/resources/iddfiles/Energy+V7_2_0.idd"
fname1 = "../eppy/resources/idffiles/V_7_2/box.idf"
# IDF.setiddname(iddfile)

In [61]:
idf = IDF(fname1)

In [62]:
surfaces = idf.idfobjects["BuildingSurface:Detailed"]
surface = surfaces[0]
print("area = %s" % (surface.area, ))
print("tilt = %s" % (surface.tilt, ))
print( "azimuth = %s" % (surface.azimuth, ))

area = 30.0
tilt = 180.0
azimuth = 0.0


Can we do the same for zones ? 

Not yet .. not yet. Not in this version on eppy

But we can still get the area and volume of the zone


In [63]:
zones = idf.idfobjects["ZONE"]
zone = zones[0]
area = modeleditor.zonearea(idf, zone.Name)
volume = modeleditor.zonevolume(idf, zone.Name)
print("zone area = %s" % (area, ))
print("zone volume = %s" % (volume, ))

zone area = 30.0
zone volume = 90.0


Not as slick, but still pretty easy

Some notes on the zone area calculation:

- area is calculated by summing up all the areas of the floor surfaces
- if there are no floors, then the sum of ceilings and roof is taken as zone area
- if there are no floors, ceilings or roof, we are out of luck. The function returns 0

## Using JSON to update idf

we are going to update `idf1` using json. First let us print the `idf1` before changing it, so we can see what has changed once we make an update

In [64]:
idf1.printidf()


VERSION,
    7.3;                      !- Version Identifier

SIMULATIONCONTROL,
    Yes,                      !- Do Zone Sizing Calculation
    Yes,                      !- Do System Sizing Calculation
    Yes,                      !- Do Plant Sizing Calculation
    No,                       !- Run Simulation for Sizing Periods
    Yes;                      !- Run Simulation for Weather File Run Periods

BUILDING,
    Empire State Building,    !- Name
    30,                       !- North Axis
    City,                     !- Terrain
    0.6,                      !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days

SITE:LOCATION,
    CHICAGO_IL_USA TMY2-94846,    !- Name
    41.78,                    !- Latitude
    -87.75,                   !- Longit

In [65]:
import eppy.json_functions as json_functions
json_str = {"idf.VERSION..Version_Identifier":8.5,
            "idf.SIMULATIONCONTROL..Do_Zone_Sizing_Calculation": "No",
            "idf.SIMULATIONCONTROL..Do_System_Sizing_Calculation": "No",
            "idf.SIMULATIONCONTROL..Do_Plant_Sizing_Calculation": "No",
            "idf.BUILDING.Empire State Building.North_Axis": 52,
            "idf.BUILDING.Empire State Building.Terrain": "Rural",
            }
json_functions.updateidf(idf1, json_str)

In [66]:
idf1.printidf()


VERSION,
    8.5;                      !- Version Identifier

SIMULATIONCONTROL,
    No,                       !- Do Zone Sizing Calculation
    No,                       !- Do System Sizing Calculation
    No,                       !- Do Plant Sizing Calculation
    No,                       !- Run Simulation for Sizing Periods
    Yes;                      !- Run Simulation for Weather File Run Periods

BUILDING,
    Empire State Building,    !- Name
    52,                       !- North Axis
    Rural,                    !- Terrain
    0.6,                      !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days

SITE:LOCATION,
    CHICAGO_IL_USA TMY2-94846,    !- Name
    41.78,                    !- Latitude
    -87.75,                   !- Longit

Compare the first printidf() and the second printidf(). 

The syntax of the json string is described below::

You can also create a new object using JSON, using the same syntax. Take a look at this:

In [67]:
json_str = {"idf.BUILDING.Taj.Terrain": "Rural",}
json_functions.updateidf(idf1, json_str)
idf1.idfobjects['building']
# of course, you are creating an invalid E+ file. But we are just playing here.

[
BUILDING,
    Empire State Building,    !- Name
    52,                       !- North Axis
    Rural,                    !- Terrain
    0.6,                      !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days
, 
BUILDING,
    Taj,                      !- Name
    0,                        !- North Axis
    Rural,                    !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days
]

What if you object name had a dot `.` in it? Will the json_function get confused?

If the name has a dot in it, there are two ways of doing this.

In [68]:
# first way
json_str = {"idf.BUILDING.Taj.with.dot.Terrain": "Rural",}
json_functions.updateidf(idf1, json_str)
# second way (put the name in single quotes)
json_str = {"idf.BUILDING.'Another.Taj.with.dot'.Terrain": "Rural",}
json_functions.updateidf(idf1, json_str)

In [69]:
idf1.idfobjects['building']

[
BUILDING,
    Empire State Building,    !- Name
    52,                       !- North Axis
    Rural,                    !- Terrain
    0.6,                      !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days
, 
BUILDING,
    Taj,                      !- Name
    0,                        !- North Axis
    Rural,                    !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days
, 
BUILDING,
    Taj.with.dot,             !- Name
    0,                        !- North Axi

**Note** When you us the json update function:

- The json function expects the `Name` field to have a value.
- If you try to update an object with a blank `Name` field, the results may be unexpected (undefined ? :-). So don't do this.
- If the object has no `Name` field (some don't), changes are made to the first object in the list. Which should be fine, since usually there is only one item in the list
- In any case, if the object does not exist, it is created with the default values

### Use Case for JSON update

If you have an eppy running on a remote server somewhere on the internet, you can change an idf file by sending it a JSON over the internet. This is very useful if you ever need it. If you don't need it, you shouldn't care :-)

## Open a file quickly¶

It is rather cumbersome to open an IDF file in eppy. From the tutorial, the steps look like this:

In [70]:
from eppy import modeleditor
from eppy.modeleditor import IDF

iddfile = "../eppy/resources/iddfiles/Energy+V7_2_0.idd"
fname = "../eppy/resources/idffiles/V_7_2/smallfile.idf"
IDF.setiddname(iddfile)
idf = IDF(fname)


- You have to find the IDD file on your hard disk.
- Then set the IDD using setiddname(iddfile).
- Now you can open the IDF file

Why can’t you just open the IDF file without jumping thru all those hoops. Why do you have to find the IDD file. What is the point of having a computer, if it does not do the grunt work for you.

The function easyopen will do the grunt work for you. It will automatically read the version number from the IDF file, locate the correct IDD file and set it in eppy and then open your file. It works like this:

In [74]:
from importlib import reload
import eppy
reload(eppy.modeleditor)
from eppy.easyopen import easyopen

fname = '../eppy/resources/idffiles/V8_8/smallfile.idf'
idf = easyopen(fname)

For this to work,

- the IDF file should have the VERSION object. You may not have this if you are just working on a file snippet.
- you need to have the version of EnergyPlus installed that matches your IDF version.
- Energyplus should be installed in the default location.

If easyopen does not work, use the long winded steps shown in the tutorial. That is guaranteed to work

## Fast HTML table file read

To read the html table files you would usually use the functions described in [Reading outputs from E+](./Outputs_Tutorial.html). For instance you would use the functions as shown below.

In [12]:
from eppy.results import readhtml # the eppy module with functions to read the html
import pprint
pp = pprint.PrettyPrinter()

fname = "../eppy/resources/outputfiles/V_7_2/5ZoneCAVtoVAVWarmestTempFlowTable_ABUPS.html" # the html file you want to read
html_doc = open(fname, 'r').read() 


htables = readhtml.titletable(html_doc) # reads the tables with their titles
firstitem = htables[0]
pp.pprint(firstitem)

('Site and Source Energy',
 [['',
   'Total Energy [kWh]',
   'Energy Per Total Building Area [kWh/m2]',
   'Energy Per Conditioned Building Area [kWh/m2]'],
  ['Total Site Energy', 47694.47, 51.44, 51.44],
  ['Net Site Energy', 47694.47, 51.44, 51.44],
  ['Total Source Energy', 140159.1, 151.16, 151.16],
  ['Net Source Energy', 140159.1, 151.16, 151.16]])


`titletable` reads all the tables in the HTML file. With large E+ models, this file can be extremeely large and `titletable` will load all the tables into memory. This can take several minutes. If you are trying to get one table or one value from a table, waiting several minutes for you reseult can be exessive.

If you know which table you are looking for, there is a faster way of doing this. We used index=0 in the above example to get the first table. If you know the index of the file you are looking for, you can use a faster function to get the table as shown below

In [18]:
from eppy.results import fasthtml
fname = "../eppy/resources/outputfiles/V_7_2/5ZoneCAVtoVAVWarmestTempFlowTable_ABUPS.html" # the html file you want to read
filehandle = open(fname, 'r') # get a file handle to the html file

In [19]:
firsttable = fasthtml.tablebyindex(filehandle, 0)
pp.pprint(firstitem)

('Site and Source Energy',
 [['',
   'Total Energy [kWh]',
   'Energy Per Total Building Area [kWh/m2]',
   'Energy Per Conditioned Building Area [kWh/m2]'],
  ['Total Site Energy', 47694.47, 51.44, 51.44],
  ['Net Site Energy', 47694.47, 51.44, 51.44],
  ['Total Source Energy', 140159.1, 151.16, 151.16],
  ['Net Source Energy', 140159.1, 151.16, 151.16]])


You can also get the table if you know the title of the table. This is the **bold** text just before the table in the HTML file. The title of our table is **Site and Source Energy**. The function `tablebyname` will get us the table.

In [17]:
filehandle = open(fname, 'r') # get a file handle to the html file
namedtable = fasthtml.tablebyname(filehandle, "Site and Source Energy")
pp.pprint(namedtable)

['Site and Source Energy',
 [['',
   'Total Energy [kWh]',
   'Energy Per Total Building Area [kWh/m2]',
   'Energy Per Conditioned Building Area [kWh/m2]'],
  ['Total Site Energy', 47694.47, 51.44, 51.44],
  ['Net Site Energy', 47694.47, 51.44, 51.44],
  ['Total Source Energy', 140159.1, 151.16, 151.16],
  ['Net Source Energy', 140159.1, 151.16, 151.16]]]


Couple of things to note here:

- We have to open the file again using `filehandle = open(fname, 'r')`
    - This is because both `tablebyname` and `tablebyindex` will close the file once they are done 
- Some tables do not have a **bold title** just before the table. `tablebyname` will not work for those functions

## Other miscellaneous functions¶

### Fan power in Watts, BHP and fan cfm¶

We normally think of fan power in terms of Brake Horsepower (BHP), Watts. Also when working with IP units it is useful to think of fan flow volume in terms of cubic feet per minute (cfm).

Energyplus does not have fields for those values. With eppy we have functions that will calculate the values

- fan power in BHP
- fan power in Watts
- fan flow in CFM

It will work for the following objects:

- FAN:CONSTANTVOLUME
- FAN:VARIABLEVOLUME
- FAN:ONOFF
- FAN:ZONEEXHAUST
- FANPERFORMANCE:NIGHTVENTILATION

The sample code would look like this:

*Note: This code was hacked together quickly. Needs peer review in ../eppy/fanpower.py*